In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("ModelRF_Deteksi_DDoS")
         .master("spark://192.168.56.9:7077")
         .getOrCreate()
        )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/09/03 10:08:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
csv_path = "hdfs://192.168.56.9:9000/user/hadoop/bigdata/dataset_sdn.csv"

df = (spark.read
      .option("header",True)
      .option("inferSchema",True)
      .csv(csv_path)
     )

In [4]:
print('5 Data Pertama :\n')
df.show(5)

print('Tipe Data" \n')
df.printSchema()

5 Data Pertama :



[Stage 3:>                                                          (0 + 1) / 1]

+-----+------+--------+--------+--------+---------+---+---------+-------+-----+---------+----------+-----------+-------+--------+--------+-------+---------+--------+-------+-------+--------+-----+
|   dt|switch|     src|     dst|pktcount|bytecount|dur| dur_nsec|tot_dur|flows|packetins|pktperflow|byteperflow|pktrate|Pairflow|Protocol|port_no| tx_bytes|rx_bytes|tx_kbps|rx_kbps|tot_kbps|label|
+-----+------+--------+--------+--------+---------+---+---------+-------+-----+---------+----------+-----------+-------+--------+--------+-------+---------+--------+-------+-------+--------+-----+
|11425|     1|10.0.0.1|10.0.0.8|   45304| 48294064|100|716000000|1.01E11|    3|     1943|     13535|   14428310|    451|       0|     UDP|      3|143928631|    3917|      0|      0|       0|    0|
|11605|     1|10.0.0.1|10.0.0.8|  126395|134737070|280|734000000|2.81E11|    2|     1943|     13531|   14424046|    451|       0|     UDP|      4|     3842|    3520|      0|      0|       0|    0|
|11425|     1|1

In [5]:
from pyspark.sql.functions import col, sum, count, lit

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

missing_df = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

+---+------+---+---+--------+---------+---+--------+-------+-----+---------+----------+-----------+-------+--------+--------+-------+--------+--------+-------+-------+--------+-----+
| dt|switch|src|dst|pktcount|bytecount|dur|dur_nsec|tot_dur|flows|packetins|pktperflow|byteperflow|pktrate|Pairflow|Protocol|port_no|tx_bytes|rx_bytes|tx_kbps|rx_kbps|tot_kbps|label|
+---+------+---+---+--------+---------+---+--------+-------+-----+---------+----------+-----------+-------+--------+--------+-------+--------+--------+-------+-------+--------+-----+
|  0|     0|  0|  0|       0|        0|  0|       0|      0|    0|        0|         0|          0|      0|       0|       0|      0|       0|       0|      0|    506|     506|    0|
+---+------+---+---+--------+---------+---+--------+-------+-----+---------+----------+-----------+-------+--------+--------+-------+--------+--------+-------+-------+--------+-----+



In [6]:
from pyspark.sql.functions import col
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import StringIndexer

df = df.withColumn("rx_kbps", col("rx_kbps").cast("double"))
df = df.withColumn("tot_kbps",col("tot_kbps").cast("double"))

imputer = Imputer(
    inputCols=["rx_kbps","tot_kbps"],
    outputCols=["rx_kbps","tot_kbps"]
).setStrategy("mean")

df = imputer.fit(df).transform(df)

Protocol_encoding = StringIndexer(inputCol="Protocol",outputCol="Protocol_encoding")
df = Protocol_encoding.fit(df).transform(df)

print("\Pemetaan dari StringIndexer:")
df.select("Protocol","Protocol_encoding").distinct().orderBy("Protocol_encoding").show()

\Pemetaan dari StringIndexer:


[Stage 17:>                                                         (0 + 1) / 1]

+--------+-----------------+
|Protocol|Protocol_encoding|
+--------+-----------------+
|    ICMP|              0.0|
|     UDP|              1.0|
|     TCP|              2.0|
+--------+-----------------+



In [7]:
from pyspark.sql.functions import col, sum, count, lit

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

missing_df = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

+---+------+---+---+--------+---------+---+--------+-------+-----+---------+----------+-----------+-------+--------+--------+-------+--------+--------+-------+-------+--------+-----+-----------------+
| dt|switch|src|dst|pktcount|bytecount|dur|dur_nsec|tot_dur|flows|packetins|pktperflow|byteperflow|pktrate|Pairflow|Protocol|port_no|tx_bytes|rx_bytes|tx_kbps|rx_kbps|tot_kbps|label|Protocol_encoding|
+---+------+---+---+--------+---------+---+--------+-------+-----+---------+----------+-----------+-------+--------+--------+-------+--------+--------+-------+-------+--------+-----+-----------------+
|  0|     0|  0|  0|       0|        0|  0|       0|      0|    0|        0|         0|          0|      0|       0|       0|      0|       0|       0|      0|      0|       0|    0|                0|
+---+------+---+---+--------+---------+---+--------+-------+-----+---------+----------+-----------+-------+--------+--------+-------+--------+--------+-------+-------+--------+-----+--------------

In [8]:
from pyspark.ml.feature import VectorAssembler

kolom_fitur = ['pktrate','pktcount','bytecount','flows','Pairflow','Protocol_encoding','tx_kbps','rx_kbps','dur','tot_dur']

assembler = VectorAssembler (inputCols=kolom_fitur,outputCol="features")
df = assembler.transform(df)


In [9]:
train_data, test_data = df.randomSplit([0.8, 0.2],seed=42)

print(f"Jumlah data train: {train_data.count()}")
print(f"Jumlah data test: {test_data.count()}")

Jumlah data train: 83587


[Stage 24:======================================>                   (2 + 1) / 3]

Jumlah data test: 20758


In [12]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol='label',featuresCol='features', numTrees=100)
model = rf.fit(train_data)

In [13]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

prediksi = model.transform(test_data)
prediksi.select('label','prediction','probability')

evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
akurasi = evaluator.evaluate(prediksi)
print(f"Akurasi model:{akurasi}")

[Stage 63:===================>                                      (1 + 2) / 3]

Akurasi model:0.924511031891319


In [20]:
model.save("rf_ddos_basemodel")

Py4JJavaError: An error occurred while calling o1294.save.
: java.io.IOException: Path rf_ddos_basemodel already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at

In [23]:
!pwd


/home/ubaid
